### Import Standard libraries

In [2]:
import pandas as pd
import numpy as np

## Method 1 using Beautiful Soup

In [3]:
# Import Beautiful Soup and requests library
import requests
from bs4 import BeautifulSoup

wiki_url = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(wiki_url,'html.parser')

In [4]:
postal_codes = []
items = soup.find('table',{'class':'wikitable sortable'}).findAll('tr')
for item in items:
    data = item.findAll(['th','td'])
    data = [x.text.strip() for x in data]
    postal_codes.append(data)
df_postal_codes = pd.DataFrame(postal_codes[1::],columns=postal_codes[0])
df_postal_codes.shape

(180, 3)

## Method 2 using wikipedia Python library

In [5]:
#Import Wikipedia library
import wikipedia as wp

html = wp.page("List_of_postal_codes_of_Canada:_M").html().encode("UTF-8")
df_postal_codes = pd.read_html(html)[0]
df_postal_codes.shape

(180, 3)

### convert all cells in the dataframe to upper case

In [6]:
df_postal_codes = df_postal_codes.apply(lambda x: x.str.upper())
df_postal_codes.head()

,Postal Code,Borough,Neighborhood
0,M1A,NOT ASSIGNED,NOT ASSIGNED
1,M2A,NOT ASSIGNED,NOT ASSIGNED
2,M3A,NORTH YORK,PARKWOODS
3,M4A,NORTH YORK,VICTORIA VILLAGE
4,M5A,DOWNTOWN TORONTO,"REGENT PARK, HARBOURFRONT"


In [7]:
df_postal_codes['Borough'].value_counts()

NOT ASSIGNED        77
NORTH YORK          24
DOWNTOWN TORONTO    19
SCARBOROUGH         17
ETOBICOKE           12
CENTRAL TORONTO      9
WEST TORONTO         6
YORK                 5
EAST YORK            5
EAST TORONTO         5
MISSISSAUGA          1
Name: Borough, dtype: int64

### cleanup Burough and Neighborhood columns as per instructions
___Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.___


___If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough.___

In [8]:
df_postal_codes = df_postal_codes[df_postal_codes['Borough'] != 'NOT ASSIGNED']
df_postal_codes.loc[(df_postal_codes['Neighborhood']=='NOT ASSIGNED')&(df_postal_codes['Borough']!='NOT ASSIGNED'), 'Neighborhood'] = df_postal_codes['Borough']
df_postal_codes.shape

(103, 3)

In [9]:
df_postal_codes.describe()

,Postal Code,Borough,Neighborhood
count,103,103,103
unique,103,10,99
top,M2N,NORTH YORK,DOWNSVIEW
freq,1,24,4


In [10]:
df_postal_codes.shape

(103, 3)

## We will be using pgeocode for Geo Location translation

In [15]:
import pgeocode
#setup Nominatim for CANADA
nomi = pgeocode.Nominatim('ca')

### Loop thru all postal codes in the Dataframe and add the Latitude and Longitude from pgeocode

In [18]:
for index, row in df_postal_codes.iterrows():
    geo_code = nomi.query_postal_code(row[0])
    df_postal_codes.loc[index,'Latitude']= geo_code['latitude']
    df_postal_codes.loc[index,'Longitude'] = geo_code['longitude']
df_postal_codes.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,NORTH YORK,PARKWOODS,43.7545,-79.3300
3,M4A,NORTH YORK,VICTORIA VILLAGE,43.7276,-79.3148
4,M5A,DOWNTOWN TORONTO,"REGENT PARK, HARBOURFRONT",43.6555,-79.3626
5,M6A,NORTH YORK,"LAWRENCE MANOR, LAWRENCE HEIGHTS",43.7223,-79.4504
6,M7A,DOWNTOWN TORONTO,"QUEEN'S PARK, ONTARIO PROVINCIAL GOVERNMENT",43.6641,-79.3889


### Check for any probable nulls

In [19]:
df_postal_codes[df_postal_codes.isnull().any(axis=1)]

,Postal Code,Borough,Neighborhood,Latitude,Longitude
114,M7R,MISSISSAUGA,CANADA POST GATEWAY PROCESSING CENTRE,NaN,NaN


### Since there are nulls, we will use the Geospatial file to update the null values with coordinates.

In [20]:
geo_data = pd.read_csv('Geospatial_Coordinates.csv')

In [21]:
for index, row in df_postal_codes[df_postal_codes.isnull().any(axis=1)].iterrows():
    df_postal_codes.loc[index, 'Latitude'] = geo_data[geo_data['Postal Code'] == row[0]].Latitude.iloc[0]
    df_postal_codes.loc[index, 'Longitude'] = geo_data[geo_data['Postal Code'] == row[0]].Longitude.iloc[0]

In [22]:
df_postal_codes[df_postal_codes.isnull().any(axis=1)]

,Postal Code,Borough,Neighborhood,Latitude,Longitude


In [24]:
df_postal_codes.head(20)

,Postal Code,Borough,Neighborhood,Latitude,Longitude
2,M3A,NORTH YORK,PARKWOODS,43.7545,-79.3300
3,M4A,NORTH YORK,VICTORIA VILLAGE,43.7276,-79.3148
4,M5A,DOWNTOWN TORONTO,"REGENT PARK, HARBOURFRONT",43.6555,-79.3626
5,M6A,NORTH YORK,"LAWRENCE MANOR, LAWRENCE HEIGHTS",43.7223,-79.4504
6,M7A,DOWNTOWN TORONTO,"QUEEN'S PARK, ONTARIO PROVINCIAL GOVERNMENT",43.6641,-79.3889
8,M9A,ETOBICOKE,"ISLINGTON AVENUE, HUMBER VALLEY VILLAGE",43.6662,-79.5282
9,M1B,SCARBOROUGH,"MALVERN, ROUGE",43.8113,-79.1930
11,M3B,NORTH YORK,DON MILLS,43.7450,-79.3590
12,M4B,EAST YORK,"PARKVIEW HILL, WOODBINE GARDENS",43.7063,-79.3094
13,M5B,DOWNTOWN TORONTO,"GARDEN DISTRICT, RYERSON",43.6572,-79.3783


In [29]:
df_postal_codes.shape

(103, 5)